In [4]:
import os
import glob

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

import gradio as gr

load_dotenv(override=True)

True

# Using langchain loaders to load dataset from local directory -> corpus.jsonl

👉 Each JSONL line is being passed as a full Python dict
👉 LangChain expects Document.page_content to be a string

In [12]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="corpus.jsonl",
    # jq_schema="._id",
    jq_schema=".text",
    json_lines=True
)

documents = loader.load()
print(f"Loaded {len(documents)} documents")


Loaded 973 documents


In [15]:
print(type(documents[0]))
documents[0].page_content[:500]  # Print the first 500 characters of the first document

<class 'langchain_core.documents.base.Document'>


'"Privileged" Nominations Every year the Senate routinely considers whether to give its advice and consent to hundreds of nominations submitted by the President. From start to finish, the confirmation process can be a lengthy one, even for relatively noncontroversial nominees. Each nomination is typically referred to one or more committees having subject matter jurisdiction over the position. Committees may bear a significant workload in examining nomineesâ\x80\x94often including questionnaires, option'

In [17]:
documents[4].metadata  # Print the metadata of the first document


{'source': 'C:\\Users\\hitan\\OneDrive\\Desktop\\me space\\Projects_RAG\\gov_docs_rag\\gov_docs_rag\\corpus.jsonl',
 'seq_num': 5}

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

textsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = textsplitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks")

Split into 61375 chunks


In [ ]:
chunks[0].page_content # Print the first chunk

'"Privileged" Nominations Every year the Senate routinely considers whether to give its advice and consent to hundreds of nominations submitted by the President. From start to finish, the confirmation process can be a lengthy one, even for relatively noncontroversial nominees. Each nomination is typically referred to one or more committees having subject matter jurisdiction over the position. Committees may bear a significant workload in examining nomineesâ\x80\x94often including questionnaires, optional public hearings, and individual meetings with Senatorsâ\x80\x94to determine whether to report a nomination to the full Senate. Once a committee has reported a nomination or been discharged from its further consideration, the Senate may take up a nomination for deliberation, though a cloture process may be required to ensure a final vote to confirm. As part of an effort to streamline the nominations process during the 112 th Congress (2011-2012), a standing order of the Senate, S.Res. 1

In [23]:
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-large-en-v1.5")

db_name = "vector_db"

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
    
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(vectorstore._collection.count())

61375
